In [22]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import keras
import re
import time
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.optimizers import Adam, SGD, Adadelta, Adagrad, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Flatten,Embedding,Conv1D, MaxPooling1D, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model, Model

from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sn

In [23]:
mymodel = load_model('/home/aogundiran/Documents/New Program/models/model-01-12-24.keras')
#mymodel = load_model('/home/aogundiran/Documents/New Program/models/model-01-08-24.keras')

In [24]:
def random_upsampling(df,label):
    majority, minority = df[label].value_counts()
    df_majority = df[df[label] == 0]
    df_minority = df[df[label] == 1]

    minority_over = df_minority.sample(majority, replace = True)
    df_over = pd.concat([df_majority, minority_over], axis = 0)
    return df_over

def random_downsampling(df,label):
    majority, minority = df[label].value_counts()
    df_majority = df[df[label] == 0]
    df_minority = df[df[label] == 1]

    majority_over = df_majority.sample(minority, replace = True)
    df_over = pd.concat([df_minority, majority_over], axis = 0)
    return df_over

# def preprocess(self,alist):
#     for i in alist:
#         split = re.split('\n',i)

#         string = "\n".join([re.sub('^\d+.','',elem) for elem in split])
#         string = re.split('(W)',string)
#         string = [re.sub(r"[\]\//[\{\}#;.&><\\\(\)'']", ' ',x) for x in string]
#         string = [x for x in string if x.strip()]
#         string = " ".join(x for x in string)
#         cleaned_code_snippet.append(string)
#     return cleaned_code_snippet

def encode(alist, vocab_size,max_length):
    encoded_docs = [one_hot(str(d),vocab_size) for d in alist]
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

def find_vocab_size(alist):
    all_words = [str(w) for s in alist for w in str(s).split()]
    print(len(all_words))
    unique_words = set(all_words)
    vocab_size = len(unique_words) + 1
    return vocab_size

In [25]:
label_encoder = preprocessing.LabelEncoder()

In [26]:
df = pd.read_csv("data/training_new.csv")
#df = pd.read_csv("data/test_new.csv")
df = df[["text", "class"]]

In [27]:
#param_df = pd.read_csv('data/vocab_params.csv')
param_df = pd.read_csv('data/vocab_params-01-12-24.csv')
#param_df = pd.read_csv('data/vocab_params-01-08-24.csv')

In [28]:
param_df.columns

Index(['vocab_size', 'max_length'], dtype='object')

In [29]:
df.head()

,text,class
0,hey!!,0
1,a little better what are u doing?,0
2,yea i thought u did i just woke and checked to...,0
3,so what did u do today?,0
4,sounds like fun lol,1


In [30]:
#df = random_upsampling(df,"class")
#df = random_downsampling(df,"class")
# code_snippet = list(df[feature])
# df[feature]=preprocess(code_snippet)

In [31]:
len(df)

903607

In [32]:
X = df
Y = np.ravel(df[["class"]])
Y = label_encoder.fit_transform(Y)

In [33]:
docs = list(X["text"])

In [34]:
vocab_size = list(param_df['vocab_size'])[0]
#print("vocab_size", vocab_size)
max_length = list(param_df['max_length'])[0]
print("max_length", max_length)
padded_docs = encode(docs,vocab_size, max_length)

max_length 9892


In [35]:
#padded_docs1 = encode(docs1, vocab_size, max_length)
test_preds = mymodel.predict(padded_docs)
test_preds = np.round(test_preds).astype(int)
test_preds1 = np.argmax(test_preds,axis=1)
accuracy = metrics.accuracy_score(Y, test_preds)
precision = metrics.precision_score(Y, test_preds)
recall = metrics.recall_score(Y, test_preds)
print ("Accuracy = ", accuracy)
print ("Precision = ", precision)
print ("Recall = ", recall)

2024-01-27 17:24:37.051579: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 35753921776 exceeds 10% of free system memory.
2024-01-27 17:25:03.958832: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.


UnknownError: Graph execution error:

Detected at node sequential/dense_5/Sigmoid defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/aogundiran/miniconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/aogundiran/miniconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/aogundiran/miniconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/aogundiran/miniconda3/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/aog

In [16]:
classification = classification_report(y_test, test_preds)
#print(classification.keys())
print("Classification report:" "\n", classification)

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    899879
           1       0.00      0.00      0.00      3728

    accuracy                           1.00    903607
   macro avg       0.50      0.50      0.50    903607
weighted avg       0.99      1.00      0.99    903607



In [ ]:
# Calculate confusion matrix
confusion_mtx = confusion_matrix( Y, test_preds, normalize= 'true')
 
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sn.heatmap(
    confusion_mtx, xticklabels=[0,1], yticklabels=[0,1], annot=True,annot_kws={"size": 10}, cmap=sn.cubehelix_palette(as_cmap=True),linewidths=.5)
plt.xlabel("Prediction")
plt.ylabel("Actual")
plt.title("Test Confusion Matrix")
plt.show()